In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cclib
import os, sys
from glob import glob

# Data preparation

In [8]:
# all output files
sys.path.append("/Users/sashaolshanova/GramNegative-Accumulation/Computational-chemistry-course-project/pKa/results_from_Dina")
refs = glob("*_pka.out")   
refs

['NH3_charged_pka.out',
 '2-3_neutral_pka.out',
 '2-20_neutral_pka.out',
 '2-2_neutral_pka.out',
 '2-46_charged_pka.out',
 '2-20_charged_pka.out',
 '2-2_charged_pka.out',
 '2-46_neutral_pka.out',
 'NH3_neutral_pka.out',
 '2-3_charged_pka.out']

In [9]:
# obtained values from output files in kcal/mol

df_pka_new = pd.DataFrame(columns=['refcode','delta G'])

for calc in refs:
  #print(calc)
  new_row = []
  data = cclib.io.ccread(calc)
  deltaG = min(data.scfenergies)*23.0621        # eV to kcal/mol 
  new_row.append(calc.split(".")[0])
  new_row.append(deltaG ) 
  df_pka_new.loc[len(df_pka_new)] = new_row
  
df_pka_new

,refcode,delta G
0,NH3_charged_pka,-3.576900e+04
1,2-3_neutral_pka,-5.205153e+05
2,2-20_neutral_pka,-2.450866e+06
3,2-2_neutral_pka,-2.426212e+06
4,2-46_charged_pka,-6.263589e+05
5,2-20_charged_pka,-2.451155e+06
6,2-2_charged_pka,-2.426504e+06
7,2-46_neutral_pka,-6.266482e+05
8,NH3_neutral_pka,-3.547934e+04
9,2-3_charged_pka,-5.208068e+05


### Final table to work with

In [10]:
#final table to work with

#charged_pka - ΔGA-
#neutral_pka - ΔGHA

short_names_pka = []
charged_neutral = []
for name in df_pka_new['refcode'].tolist():
    #print(name)
    x = name.split("_", 1)[0]
    charged_neutrall = name.split("_", 1)[1]
    short_names_pka.append(x)
    charged_neutral.append(charged_neutrall)
df_pka_new['name'] =  short_names_pka
df_pka_new['Charge'] =  charged_neutral

table = pd.pivot_table(df_pka_new, values ='delta G', index =['name'],
                         columns =['Charge'])

#format column names
table.columns = [''.join(str(s).strip() for s in col if s) for col in table.columns]

#reset index
table.reset_index(inplace=True)

table

,name,charged_pka,neutral_pka
0,2-2,-2.426504e+06,-2.426212e+06
1,2-20,-2.451155e+06,-2.450866e+06
2,2-3,-5.208068e+05,-5.205153e+05
3,2-46,-6.263589e+05,-6.266482e+05
4,NH3,-3.576900e+04,-3.547934e+04


# Calculations

## Easy way to calculate pKa

In [11]:
T = 298.150 
R = 1.9872036e-03 
H_const = -265.6
list_pkaa = []

for compound in table['name'].tolist():
    #print(compound)
    delta_charged = table[table['name'] == compound]['charged_pka'].tolist()[0]
    delta_neutral = table[table['name'] == compound]['neutral_pka'].tolist()[0]
    pka = (delta_charged+H_const-delta_neutral) / (R*T*2.3)                        # important part for calculations 
    list_pkaa.append(pka)
table['pKa_easy_way'] = list_pkaa 
table  

,name,charged_pka,neutral_pka,pKa_easy_way
0,2-2,-2.426504e+06,-2.426212e+06,-409.626542
1,2-20,-2.451155e+06,-2.450866e+06,-406.674503
2,2-3,-5.208068e+05,-5.205153e+05,-408.839644
3,2-46,-6.263589e+05,-6.266482e+05,17.349473
4,NH3,-3.576900e+04,-3.547934e+04,-407.462278


## Calculations based on the final equation

In [12]:
# part of the equation related to the NH3

T = 298.150 
R = 1.9872036e-03
pkbNH3 = 4.76
deltaG_NH33 = table[table['name'] == 'NH3']['charged_pka'].tolist()[0] - table[table['name'] == 'NH3']['neutral_pka'].tolist()[0]

table = table.drop([table[table['name'] == 'NH3'].index.values.tolist()[0]])
table

,name,charged_pka,neutral_pka,pKa_easy_way
0,2-2,-2.426504e+06,-2.426212e+06,-409.626542
1,2-20,-2.451155e+06,-2.450866e+06,-406.674503
2,2-3,-5.208068e+05,-5.205153e+05,-408.839644
3,2-46,-6.263589e+05,-6.266482e+05,17.349473


In [16]:
pka = []
for compound in table['name'].tolist():
    deltaG_sum = table[table['name'] == compound]['charged_pka'].tolist()[0] - table[table['name'] == compound]['neutral_pka'].tolist()[0] + deltaG_NH33
    pka_value = deltaG_sum/(R*T*2.3) - pkbNH3
    pka.append(pka_value)
table['PKA'] = pka
table

,name,charged_pka,neutral_pka,pKa_easy_way,PKA
0,2-2,-2.426504e+06,-2.426212e+06,-409.626542,-432.038753
1,2-20,-2.451155e+06,-2.450866e+06,-406.674503,-429.086713
2,2-3,-5.208068e+05,-5.205153e+05,-408.839644,-431.251855
3,2-46,-6.263589e+05,-6.266482e+05,17.349473,-5.062738


In [17]:
table.rename({'charged_pka': 'ΔGA-', 'neutral_pka': 'ΔGHA' }, axis='columns')

,name,ΔGA-,ΔGHA,pKa_easy_way,PKA
0,2-2,-2.426504e+06,-2.426212e+06,-409.626542,-432.038753
1,2-20,-2.451155e+06,-2.450866e+06,-406.674503,-429.086713
2,2-3,-5.208068e+05,-5.205153e+05,-408.839644,-431.251855
3,2-46,-6.263589e+05,-6.266482e+05,17.349473,-5.062738
